In [ ]:
%%cu
#include<iostream>
#include<cstdlib>
using namespace std;

//Kernel Functions
template <typename T>
__global__ void vectorAdd(T *a, T *b, T *result, int n) {
  int tid = blockIdx.x*blockDim.x + threadIdx.x;
  result[tid] = a[tid] + b[tid];
}

template <typename T>
__global__ void matrixMultiplication(T *a, T *b, T *c, int m, int n, int k){
  int row = blockIdx.y*blockDim.y + threadIdx.y;
  int col = blockIdx.x*blockDim.x + threadIdx.x;
  T sum=0;
  if(col<k && row<m) {
    for(int j=0;j<n;j++){
      sum += a[row*n+j] * b[j*k+col];
    }
    c[k*row+col]=sum;
  }
}

template <typename T>
__global__ void matrixVector(T *vec, T *mat, T *result, int n, int m) {
  int tid = blockIdx.x*blockDim.x + threadIdx.x;
  T sum=0;
  for(int i=0; i<n; i++) {
    sum += vec[i]*mat[(i*m) + tid];
  }
  result[tid] = sum;
}

//Classes
template <class T>
class Tensor{
  public:
  virtual void initialize_random() const=0;
  virtual T* getArr() const=0;
};

template <class T>
class Matrix : public Tensor<T>{

  private:
  T* arr;
  int rows, cols;

  public:
  Matrix(int r, int c, bool init_rand=true){
    rows = r;
    cols = c;
    arr = new T[rows*cols];
    if(init_rand==true){
      initialize_random();
    }
  }

  void initialize_random() const {
    for(int i=0; i<rows; i++) {
      for(int j=0; j<cols; j++) {
        arr[i*cols + j] = rand()%10 + 1;
      }
    }
  }

  T* getArr() const {
    return arr;
  }

  int getRows() const {
    return rows;
  }

  int getCols() const{
    return cols;
  }

  Matrix operator* (const Matrix& arg){
    
    //Declaring Device Variables
    int *a_dev,*b_dev,*c_dev;
 
    int m=rows, n=cols, k=arg.cols;
 
    Matrix c(m,k,false);
    
    //Allocating Memory to Device Variables


    cudaMalloc(&a_dev, sizeof(T)*m*n);
    cudaMalloc(&b_dev, sizeof(T)*n*k);
    cudaMalloc(&c_dev, sizeof(T)*m*k);
    
    //Copying Mmmory from CPU to GPU (operands)
    
    cudaMemcpy(a_dev, arr, sizeof(T)*m*n, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, arg.arr, sizeof(T)*n*k, cudaMemcpyHostToDevice);
    
    int dimb = (m>n)?m:n;
    dimb = (k>dimb)?k:dimb;
    dim3 dimGrid(1,1);
    dim3 dimBlock(dimb,dimb);
    matrixMultiplication<<<dimGrid, dimBlock>>>(a_dev,b_dev,c_dev, m, n, k);

    //source,dest,size
     
    cudaMemcpy(c.arr, c_dev, sizeof(T)*m*k, cudaMemcpyDeviceToHost);
    
    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);
    
    return c;
      
  }

  //Overloading insertion operator to display Vector
  //Friend Function
  template <typename U>
  friend ostream& operator<<(ostream& os, const Matrix <U> &matrix);

};

template <typename T>
ostream& operator<<(ostream& os, const Matrix<T> &matrix){
  os<<endl;
  for(int i=0; i<matrix.rows; i++) {
    os<<"["<<matrix.arr[i*matrix.cols];
    for(int j=1; j<matrix.cols; j++){
      os<<",\t"<<matrix.arr[i*matrix.cols + j];
    }
    os<<"]"<<endl;
  }
  return os;
}


template<class T>
class Vector : public Tensor<T> {
    
  //Private Variables
  private:
  T *arr;
  int length;

  //Public Methods
  public:

  //Constructor with Default Parameter init_rand
  Vector(int n, bool init_rand=true){
    length = n;
    arr = (T*)malloc(n * sizeof(T));
    if(init_rand==true){
      initialize_random();
    }
  }

  //Initializes Elements to a random value
  void initialize_random() const{
    for(int i=0; i<length; i++) {
      arr[i] = rand()%10 * 1.1;
    }
  }

  //Returns number of array elements
  int getLength() const {
    return length;
  }

  T* getArr() const {
    return arr;
  }

  //Overloading + operator for addition of two vectors
  Vector operator+(const Vector& arg) {
      
    if(length!=arg.length){
      return Vector(0);
    }

    Vector result(length, false);

    //Device Variables
    T *this_dev, *arg_dev, *result_dev;

    int size = length * sizeof(T);

    //Allocating Memory to Device Variables
    cudaMalloc(&this_dev, size);
    cudaMalloc(&arg_dev, size);
    cudaMalloc(&result_dev, size);

    //Copying Variables from Host to Device
    cudaMemcpy(this_dev, arr, size, cudaMemcpyHostToDevice);
    cudaMemcpy(arg_dev, arg.arr, size, cudaMemcpyHostToDevice);

    //Kernel Launch
    vectorAdd<<<1,length>>>(this_dev, arg_dev, result_dev, length);

    //Copying Variables from Device to Host
    cudaMemcpy(result.arr, result_dev, size, cudaMemcpyDeviceToHost);

    //Freeing Device Memory
    cudaFree(this_dev);
    cudaFree(arg_dev);
    cudaFree(result_dev);

    return result;

  }


  //Overloading * operator for product of vector and matrix
  Vector operator*(const Matrix<T> &arg) {

    int *a_dev, *b_dev, *c_dev;
    
    int n = length;
    int m = arg.getCols();

    Vector c(m,false);

    cudaMalloc(&a_dev, sizeof(int)*n);
    cudaMalloc(&b_dev, sizeof(int)*n*m);
    cudaMalloc(&c_dev, sizeof(int)*m);

    T* arg_arr = arg.getArr();
    cudaMemcpy(a_dev, arr, sizeof(int)*n, cudaMemcpyHostToDevice);
    cudaMemcpy(b_dev, arg_arr, sizeof(int)*n*m, cudaMemcpyHostToDevice);

    int dimt = (m>n)?m:n;

    matrixVector<<<1, dimt>>>(a_dev, b_dev, c_dev, n, m);

    cudaMemcpy(c.arr, c_dev, sizeof(int)*m, cudaMemcpyDeviceToHost);

    cudaFree(a_dev);
    cudaFree(b_dev);
    cudaFree(c_dev);

    return c;

  }

  //Overloading insertion operator to display Vector
  //Friend Function
  template <typename U>   
  friend ostream& operator<<(ostream& os, const Vector<U> &vec);

};

template <typename T>
ostream& operator<<(ostream& os, const Vector<T> &vec){
  if(vec.length==0){
    os<<"[ ]";
    return os;
  }
  os<<"["<<vec.arr[0];
  for(int i=1; i<vec.length; i++) {
    os<<","<<vec.arr[i];
  }
  os<<"]";
  return os;
}

//Assignment Subparts

void vectorAdditionInt(){
 
  cout<<"\n=================================\n";
  cout<<"Vector Addition (int)\n";

  //Vector of type int
  Vector<int> a(8), b(8), c=a+b;
  cout<<"\nA = "<<a;
  cout<<"\nB = "<<b;
  cout<<"\nC = "<<c;
  cout<<endl;
    
}

void vectorAdditionFloat(){
 
  cout<<"\n=================================\n";
  cout<<"Vector Addition (float)\n";

  //Vector of type float
  Vector<float> d(3), e(3), f=d+e;
  cout<<"\nD = "<<d;
  cout<<"\nE = "<<e;
  cout<<"\nF = "<<f;
  cout<<endl;
    
}

void vectorMatrixMultiplication(){
 
  cout<<"\n=================================\n";
  cout<<"Vector and Matrix Multiplication\n";
 
  Vector<int> v(3);
  Matrix<int> mat(3,4);
  Vector<int> res = v*mat;
  cout<<"\nVector = "<<v;
  cout<<"\n\nMatrix : "<<mat;
  cout<<"\nResult = "<<res;
  cout<<endl;
    
}

void MatricesMultiplication(){
 
  cout<<"\n=================================\n";
  cout<<"Matrix Multiplication\n";

  int m=5, k=4, n=3;
  Matrix<int> m1(m,k),m2(k,n),m3=m1*m2;
  cout<<"\nMatrix A : "<<m1;
  cout<<"\nMatrix B : "<<m2;
  cout<<"\nResultant Matrix : "<<m3;
    
}

int main(){
 
  vectorAdditionInt();
  vectorAdditionFloat();
  vectorMatrixMultiplication();
  MatricesMultiplication();

  return 0;
}


Vector Addition (int)

A = [3,6,7,5,3,5,6,2]
B = [9,1,2,7,0,9,3,6]
C = [12,7,9,12,3,14,9,8]

Vector Addition (float)

D = [0,6.6,2.2]
E = [6.6,1.1,8.8]
F = [6.6,7.7,11]

Vector and Matrix Multiplication

Vector = [7,9,2]

Matrix : 
[1,	3,	4,	8]
[6,	10,	3,	3]
[9,	10,	8,	4]

Result = [79,131,71,91]

Matrix Multiplication

Matrix A : 
[7,	2,	3,	10]
[4,	2,	10,	5]
[8,	9,	5,	6]
[1,	4,	7,	2]
[1,	7,	4,	3]

Matrix B : 
[1,	7,	2]
[6,	6,	5]
[8,	7,	6]
[7,	10,	4]

Resultant Matrix : 
[113,	182,	82]
[131,	160,	98]
[144,	205,	115]
[95,	100,	72]
[96,	107,	73]

